In [33]:
!pip install -U "langchain[google-genai]"

  Using cached langchain_google_genai-3.0.0-py3-none-any.whl.metadata (7.1 kB)
  Using cached google_ai_generativelanguage-0.9.0-py3-none-any.whl.metadata (10 kB)
  Using cached filetype-1.2.0-py2.py3-none-any.whl.metadata (6.5 kB)
Using cached langchain_google_genai-3.0.0-py3-none-any.whl (57 kB)
Using cached filetype-1.2.0-py2.py3-none-any.whl (19 kB)
Using cached google_ai_generativelanguage-0.9.0-py3-none-any.whl (1.4 MB)


In [34]:
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model


load_dotenv()
model = init_chat_model("google_genai:gemini-2.5-flash-lite")




In [35]:

from langchain.agents import create_agent



def get_weather(city : str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}"

agent = create_agent(
    model=model,
    tools = [get_weather],
    system_prompt="You are a helpful assistant"
)

agent.invoke(
    {"messages" : [{
        "role" : "user",
        "content" : "what is the weather in sf"
        
        }]
    }
)

{'messages': [HumanMessage(content='what is the weather in sf', additional_kwargs={}, response_metadata={}, id='8fbeccca-8cee-4bf9-8119-77ed7d64cda3'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_weather', 'arguments': '{"city": "sf"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--d5aaeb4f-8fac-41e3-b615-9f513ace6160-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'sf'}, 'id': '88881c0b-0527-4f91-8c93-d937c7ad5da4', 'type': 'tool_call'}], usage_metadata={'input_tokens': 51, 'output_tokens': 15, 'total_tokens': 66, 'input_token_details': {'cache_read': 0}}),
  ToolMessage(content="It's always sunny in sf", name='get_weather', id='a4fe840a-10e5-4307-bbfd-485b9651c437', tool_call_id='88881c0b-0527-4f91-8c93-d937c7ad5da4'),
  AIMessage(content="The weather i

In [37]:
from dataclasses import dataclass


@dataclass
class ResponseFormat:
    """Response schema for the agent."""
    punny_response: str
    weather_conditions: str | None = None

@dataclass
class Context:
    """Custom runtime context schema"""
    user_id : str

In [ ]:
from langchain.tools import tool, ToolRuntime


@tool
def get_weather_for_location(city : str) -> str:
    """Get weather for a given city."""



@tool
def get_user_location(runtime : ToolRuntime[Context]) -> str:
    """Retrieve user information based on User Id."""
    user_id = runtime.context.user_id
    return "Florida" if user_id == "1" else "SF"

In [ ]:
from langgraph.checkpoint.memory import InMemorySaver

checkpointer = InMemorySaver()

In [ ]:
agent = create_agent(
    model=model,
    system_prompt="",
    tools=[get_user_location, get_weather_for_location],
    context_schema=Context,
    response_format=ResponseFormat,
    checkpointer=checkpointer
)

config = {"configurable" : {"thread_id" : "1"}}

response = agent.invoke(
    {
        "messages" : [{
                "role" : "user",
                "content" : "what is the weather outside?"
            }],
    },
    config=config,
    context=Context(user_id="1")
)

print(response['structured_response'])